# The Wealth of Cities
## Predicting the Wealth of a City from Satellite Imagery

Accurate measurements of the economic characteristics of populations critically influence research and policy. Such measurements can shape decisions by governments and how to allocate resources and provide infrastructure to improve human livelihoods in a wide-range of situations. Although economic data is readily available for some developing nations, many regions of the modern world remain unexposed to the benefits of economic analysis where regions lack key measures of economic development and efficiency. Regions such as parts of Africa conduct little to no economic surveys or other means of collecting data on their financial situations. We attempt to address this problem by using publicly available satellite imagery to predict the wealth of a city (or, more generally, a geographic region) based on fundamental features identified in these images and running them through a convolutional neural network. Not only would this method be applicable to regions that lack economic data, but could also be applied to cities with a wealth of economic information on a macro level but a dearth on a micro level. For example, cities in America, despite having lots of economic data on state and county levels, could benefit from understanding more granular information in order to improve policy decisions for infrastructure and public support. 

In order for this approach to work, we need to be able to extract relevant features from the images in order to train our machine learning model. Our model will not be able to predict the wealth of individual houses (i.e., families), but will work on clusters of houses (i.e., neighborhoods) because of the complexity of wealth measurements and tendency for neighborhood to be at a nearly homogeneous economic level. As a result, we will need to extract "cluster" features to process with our (NEURAL NETWORK).

Thinking about the kinds of features that would elucidate the wealth of a region, we can start to identify what we need to extract (in some way) from the images. One of the first and most common thoughts is to get satellite imagery of the region at night and observe the night-light intensity; more lights at night tend to correspond with more wealth while less lights at night tend to correspond with poorer areas. Our group has also thought of the following ideas as means to identify wealth:
- Number of cars
- Percentage of green-space
- Number of high-rises
- What time traffic occurs at
- Housing density
- Aerospace/nautical infrastructure

The number of of cars tends to be a good indicator or whether a city has passed a certain threshold for wealth. Yes, some cities that are poorer than others will have more cars, but cities that have no cars tend to be the poorest, so we can figure out a baseline level for the wealth of a city if we can extract the number of cars from the image.

Percentage of green space is perhaps even less reliable than the number of cars, but can also establish relative rankings of wealth between multiple cities. Cities with lots of public funds, and consequently wealth, will tend to spend money on maintaining public green-spaces. Granted, some rural regions tend to also have a lot of green-space in the form of farms or undeveloped land, so in this case green-space does not correspond to higher wealth. However, if we can ensure that the imagery we are looking at represents a urban city, we could perhaps take into  green-space into account to predict the level of wealth.

Number of high-rises is definitely a critical feature of a city's wealth. However, extracting this information from satellite imagery proves to be tricky because of the flatness of the images. One way to get around this is to analyze the shadows produced by buildings at different times of the day. If the buildings are tall, they will cast long shadows at all times of the day (not only briefly in the morning and night).

Housing density is highly correlated to the "urban-ness" of a region, which in turn is suggestive of the wealth of a city. Rural areas (i.e., poorer, generally) have a lower housing density while urban areas (i.e., wealthier, generally) have a higher housing density. Granted, there are exceptions to this trend, but generally this fact will hold and is one of the easier features to extract from satellite imagery.

We will be getting our images from Planet.com, a publicly available database of satellite imagery from the last few years that covers most of the world. Unfortunately, API access is limited to California so we will only be able to run our model using data from California, but there is no reason that this method would not work given more input data from around the world.

In this notebook, we'll take you through the entire process from setting up the program to download images and extract features to running the data through the machine learning pipeline and getting a predicted wealth score for input data. 

First, we'll input the necessary modules. `json` and `io` are just used to load in our Planet.com API key. You can sign up for a free account at https://www.planet.com/. The approval process will take a few days, but after receiving your API key, this entire notebook can be completed in one sitting. We will be using the `requests` module to make API requests for the satellite imagery, which requires authorization using the `requests.auth` module.

In [4]:
import numpy as np
import json, io, math
import requests
from requests.auth import HTTPBasicAuth
from PIL import Image
from PIL import ImageColor
from scipy.misc import toimage
import scipy.ndimage
import cv2

In [5]:
# LOADS in your API_KEY from the config_secret.json file
with io.open("config_secret.json") as cred:
    API_KEY = json.load(cred)["API_KEY"]

BASE_URL = "https://maps.googleapis.com/maps/api/staticmap"
MODES = ["road", "satellite"]
MAX_RGB = 255
DEFAULT_SIZE = 600
DEFAULT_VISIBILITY = "off"

CAR_EDGE_TEMPLATE = cv2.Canny(cv2.cvtColor(np.asarray(Image.open("car_template.png")), cv2.COLOR_BGR2GRAY), 10, 300)

# (lat/pixel, lon/pixel)
# Multiply by pixels to see lat/lon span of image
# Only want zoom levels 13-20
# City zoom = 12
ZOOMS = {
    12: (0.06/230, 0.06/175),
    13: (0.02/153, 0.02/116),
    14: (0.008/122, 0.008/122),
    15: (0.008/245, 0.008/187),
    16: (0.003/183, 0.003/140),
    17: (0.002/245, 0.002/186),
    18: (0.001/245, 0.001/186),
    19: (0.0005/245, 0.0005/186),
    20: (0.0002/196, 0.0002/149)
}

In [6]:
EARTH_RADIUS = 6371000
BIGIMAGE_W, BIGIMAGE_H = (54000, 27000)
OCTANTIMAGE_W, OCTANTIMAGE_H = (13500, 13500)
BIG_IMAGE = np.asarray(Image.open("night_images/dnb_land_ocean_ice_geo.tif"))
OCTANT_IMAGES = [np.asarray(Image.open("night_images/dnb_land_ocean_ice.{}_geo.tif".format(i+1))) for i in xrange(8)]
OCTANT_RATIOS = [(0, 0), (BIGIMAGE_W/4.0, 0), (BIGIMAGE_W/2.0, 0), (3*BIGIMAGE_W/4.0, 0),
                 (0, BIGIMAGE_H/2.0), (BIGIMAGE_W/4.0, BIGIMAGE_H/2.0), (BIGIMAGE_W/2.0, BIGIMAGE_H/2.0), 
                 (3*BIGIMAGE_W/4.0, BIGIMAGE_H/2.0)]
OCTANTS = zip(OCTANT_IMAGES, OCTANT_RATIOS)

In [7]:
def load_image(content):
    return Image.open(io.BytesIO(content)).convert("RGBA").convert("RGB")

def rgb_to_hex(rgb_tuple):
    return "0x%02x%02x%02x" % rgb_tuple

def create_payload(mode, (lat, lon), zoom, params={}, ret_colors=True):

    size = params.get("size", (DEFAULT_SIZE, DEFAULT_SIZE))
    padding = params.get("padding", 0)
    road_color = params.get("road_color", (0, MAX_RGB, 0))
    road_color_hex = rgb_to_hex(road_color)
    man_made_color = params.get("man_made_color", (0, 0, 0))
    man_made_color_hex = rgb_to_hex(man_made_color)
    poi_color = params.get("poi_color", (MAX_RGB, 0, 0))
    poi_color_hex = rgb_to_hex(poi_color)
    water_color = params.get("water_color", (0, 0, MAX_RGB))
    water_color_hex = rgb_to_hex(water_color)
    natural_color = params.get("natural_color", (MAX_RGB, 0, MAX_RGB))
    natural_color_hex = rgb_to_hex(natural_color)
    label_visibility = params.get("label_visibility", DEFAULT_VISIBILITY)

    base_payload = [("size", "{}x{}".format(size[0],size[1])), ("key", API_KEY)]
    style_payload = [("style", "feature:road|element:geometry|color:{}|visibility:on".format(road_color_hex)),
                     ("style", "feature:landscape.man_made|element:geometry.fill|color:{}".format(man_made_color_hex)),
                     ("style", "element:labels|visibility:{}".format(label_visibility)),
                     ("style", "feature:poi|element:geometry|color:{}".format(poi_color_hex)),
                     ("style", "feature:water|element:geometry|color:{}".format(water_color_hex)),
                     ("style", "feature:landscape.natural|element:geometry.fill|color:{}".format(natural_color_hex))]
    satellite_payload = base_payload + [("maptype", "satellite")]
    road_payload = base_payload + style_payload + [("maptype", "roadmap")]
    
    if mode == "satellite": payload = satellite_payload 
    elif mode == "road": payload = road_payload
    else: raise ValueError("Unrecognized mode '{}'. Mode can either be 'satellite' or 'road'.".format(mode))
        
    payload += [("zoom", zoom)] + [("center", "{},{}".format(lat, lon))]
    colors = {
        "road": np.array(road_color),
        "man_made": np.array(man_made_color), 
        "poi": np.array(poi_color),
        "water": np.array(water_color),
        "natural": np.array(natural_color)
    }
    
    return (payload, colors) if ret_colors else payload
    

# bottom left, top right corners
def bounding_box((lat1,lon1), (lat2,lon2), zoom):
    w = lon2 - lon1
    h = lat2 - lat1
    
    w_per_image = DEFAULT_SIZE * ZOOMS[zoom][1]
    h_per_image = DEFAULT_SIZE * ZOOMS[zoom][0]
    
    num_width = math.ceil(w / w_per_image)
    num_height = math.ceil(h / h_per_image)
    
    lons = np.linspace(lon1 + w_per_image/2, lon2 - w_per_image/2, num=num_width)
    lats = np.linspace(lat1 + h_per_image/2, lat2 - h_per_image/2, num=num_height)
    
    return lats, lons

def get_image(lat, lon, payload, zoom):
    r = requests.get(BASE_URL, params=payload)
    image = load_image(r.content)
    return image

def get_images((lat1,lon1), (lat2,lon2), zoom, modes, ret_colors=True):
    lats, lons = bounding_box((lat1,lon1), (lat2,lon2), zoom)
    num_images = len(lats) * len(lons)
    print "Downloading {} images...".format(num_images)
    images = {
        "road": [],
        "satellite": []
    }
    
    i = 1
    for lat in lats:
        for lon in lons:
            for mode in modes:
                print "\tDownloading image " + i
                i += 1
                payload, colors = create_payload(mode, (lat, lon), zoom)
                images[mode].append( get_image(lat, lon, payload, zoom) )
                
    print "Done downloading images!"        
    return (images, num_images, colors) if ret_colors else (images, num_images)

In [8]:
def find_loc(lat, lon, radius):
    y = np.sqrt(2*radius**2 * (1 - np.cos(lat)))
    x = np.sqrt(2*radius**2 * (1 - np.cos(lon)))
    return x, y

def find_pixel(lat, lon, radius, scale):
    x, y = find_loc(lat, lon, radius)
    centerx = BIGIMAGE_W / 2
    centery = BIGIMAGE_H / 2
    return centerx + x/scale, centery + y/scale

def find_octant(latR, lonR):
    if (lonR < 0.5):
        if (latR < 0.25):
            return 0
        elif (latR < 0.5):
            return 1
        elif (latR < 0.75):
            return 2
        else: 
            return 3
    else:
        if (latR < 0.25):
            return 4
        elif (latR < 0.5):
            return 5
        elif (latR < 0.75):
            return 6
        else:
            return 7

def find_light(x, y, bigimage, octants):
    octant_num = find_octant(x / BIGIMAGE_W, y / BIGIMAGE_H)
    octantimage, (ox, oy) = octants[octant_num]
    x2 = OCTANTIMAGE_W * (x-ox) / (BIGIMAGE_W/4.0) 
    y2 = OCTANTIMAGE_H * (y-oy) / (BIGIMAGE_H/2.0)
    return octantimage[x2][y2]

def average_light(lat, lon):
    pixelx, pixely = find_pixel(lat, lon, EARTH_RADIUS, 750)
    pixels = [(pixelx, pixely), (pixelx + 1, pixely),(pixelx - 1, pixely), (pixelx, pixely - 1), (pixelx, pixely + 1)]
    return np.mean([find_light(x, y, BIG_IMAGE, OCTANTS) for (x, y) in pixels])

In [8]:
def get_pixels_of_color(im_arr, color, tolerance=10):
    lower_bound = color - tolerance
    lower_bound[lower_bound < 0] = 0
    upper_bound = color + tolerance
    upper_bound[upper_bound > MAX_RGB] = MAX_RGB
    return np.all((im_arr >= lower_bound) & (im_arr <= upper_bound), axis=2)

def find_roads(road_arr, road_color, tolerance=10):
    return get_pixels_of_color(road_arr, road_color, tolerance=tolerance)

def road_variance(sat_arr, road_only_arr):
    sat_roads = sat_arr[road_only_arr]
    return sum(np.std(sat_roads, axis=1))

def count_object_pixels(img_rgb, obj_edge_template=CAR_EDGE_TEMPLATE, threshold=0.1, angular_granularity=1):
    img_rgb = np.asarray(img_rgb)
    w, h = obj_edge_template.shape
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.Canny(img_gray, 100, 200)
    loc = (np.array(0), np.array(0))
    
    for i in range(angular_granularity):
        template = scipy.ndimage.rotate(obj_edge_template, i * 360. / angular_granularity, mode="constant")
        match_coeff = cv2.matchTemplate(img_edges, template, cv2.TM_CCOEFF_NORMED)
        found = np.where(match_coeff > threshold)
        loc = (np.append(loc[0], found[0]), np.append(loc[1], found[1]))
    
    return len(set(zip(*loc)))

def extract_features(road_arr, sat_arr, colors, ((lat1, lon1), (lat2, lon2)), tolerance=10):
    total_pixels = road_arr.shape[0] * road_arr.shape[1]
    road_pixels = {}
    road_pixel_counts = {}
    
    for kind, color in colors.iteritems():
        road_pixels[kind] = get_pixels_of_color(road_arr, color, tolerance=tolerance)
        
    for kind, color_pixels in road_pixels.iteritems():
        road_pixel_counts[kind] = np.sum(color_pixels)
        
    road_var = road_variance(sat_arr, road_pixels["road"])
    
    avg_light = average_light(lat1 + (lat2 - lat1)/2, lon1 + (lon2 - lon1)/2)
    
    feature1 = np.array(road_pixels.items())
    feature2 = np.array(road_pixel_counts.items()) / total_pixels
    feature3 = np.array([road_var])
    feature4 = np.array([avg_light])
    print feature1
    print feature2
    print feature3
    print feature4
    print np.concatenate(feature1, feature2, feature3, feature4)

In [2]:
def get_features_for_city(((lat1, lon1), (lat2, lon2))):
    zoom = 13
    modes = MODES
    images, n, colors = get_images((lat1, lon1), (lat2, lon2), 12, modes)
    
#     for i in xrange(1):
#         road_arr = np.asarray(images["road"][i])
#         sat_arr = np.asarray(images["satellite"][i])
#         extract_features(road_arr, sat_arr, colors, ((lat1, lon1), (lat2, lon2)))

NAMES = ["Pittsburgh, PA", "San Fransisco, CA", "New York, NY", "DC", "Blackwater, AZ", "Sneedsville, TN"]
POINTS = [((40.417268, -80.036749), (40.418268, -80.035749)),
          ((37.710978, -122.500087), (37.802606, -122.404110)),
          ((40.510705, -74.252698), (40.867683, -73.764113)),
          ((38.819597, -77.160145), (38.983113, -76.912953)),
          ((33.000521, -111.664691), (33.045859, -111.568733)),
          ((36.513680, -83.234828), (36.555887, -83.166506))]

get_features_for_city(POINTS[0])

# FEATURES = [get_features_for_city(city) for city in points]
LABELS = [63024, 59799, 57291, 9491, 13719]
# clf = svm.SVR()
# clf.fit(features, labels)
# clf.predict(test_features)

NameError: global name 'MODES' is not defined

In [ ]:
### TESTS ###
# zoom = 13 -> 6 images (1.5 seconds)
# PGH = ((40.417268, -80.036749), (40.503523, -79.823013), 13, MODES)
# zoom = 20 -> 4 images (1 second)
# pgh_images, n, colors = get_images((40.417268, -80.036749), (40.418268, -80.035749), 20, MODES)

To get an idea of what these satellite images look like, we will show you how to download a single image and then proceed to, what Planet calls, an Area of Interest, or AOI. First, we define a geometry, which is a collection of latitude and longitude points  that forms a polygon around the area you would like to get pictures from. Remember that Planet API only works with California right now, so if you want to change the coordinates, make sure they remain within the state. Our example geometry is centered on a reservoir in Redding, CA. Next, we'll need to define filters for the Planet API; these include the geometry filter discussed above, as well as date range filters (only getting images within a specified date range), cloud cover filters (perhaps you only want to look at images on clear day), and many more. We then send this request to the Stats API endpoint to see how many possible images there are that fit our criteria. In our example, there are 30 images taken of Redding, CA within the date range that have less than 50% cloud cover.